In [ ]:
# Step 1: Load CSV File
import pandas as pd

def load_csv(file_path):
    """Load a CSV file and return a DataFrame."""
    try:
        data = pd.read_csv(file_path)
        print("CSV loaded successfully!")
        return data
    except Exception as e:
        print(f"Error loading CSV: {e}")
        return None

# Test loading the CSV
csv_file = "sample_integration_data.csv"  # Replace with your actual CSV path
data = load_csv(csv_file)

# Drop the 'Context-Domain' column
data = data.drop(columns=['Context-Domain'], errors='ignore')

# Display the first few rows to confirm the column is dropped
print("DataFrame after dropping 'Context-Domain':")

# Display the first few rows of the data
if data is not None:
    print(data.head())

In [ ]:
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, PointStruct

# Initialize SentenceTransformer model
vectorizer = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Set up Qdrant
def setup_vector_db(data):
    """
    Create and populate an in-memory Qdrant vector database with graph data.
    """
    try:
        # Initialize Qdrant client (in-memory)
        client = QdrantClient(":memory:")
        
        # Define vector collection schema
        collection_name = "systems"
        if client.get_collections().collections:
            client.delete_collection(collection_name)
        
        client.create_collection(
            collection_name=collection_name,
            vectors_config=VectorParams(size=384, distance="Cosine")
        )

        # Populate the database
        points = []
        for idx, row in data.iterrows():
            # Create a descriptive text for the relationship
            interaction_text = f"{row['Consumer']} interacts with {row['Producer']} via {row['Integration Type']}"
            
            # Encode text into a vector
            embedding = vectorizer.encode(interaction_text).tolist()
            
            # Add point with metadata
            points.append(
                PointStruct(
                    id=idx,  # Unique ID
                    vector=embedding,
                    payload={
                        "consumer": row['Consumer'],
                        "producer": row['Producer'],
                        "integration": row['Integration Type']
                    }
                )
            )
        
        # Upsert points into Qdrant
        client.upsert(collection_name=collection_name, points=points)
        print(f"Vector database populated with {len(points)} points!")
        return client
    except Exception as e:
        print(f"Error setting up vector database: {e}")
        return None

# Set up the vector database
vector_db_client = setup_vector_db(data)

In [ ]:
def query_vector_db(client, query, role="consumer", top_k=5):
    """
    Query the Qdrant vector database for relevant interactions.
    
    Args:
        client (QdrantClient): Qdrant client instance.
        query (str): Search query (e.g., "HR system").
        role (str): Specify whether to prioritize as 'consumer' or 'producer'.
        top_k (int): Number of top results to retrieve.
    
    Returns:
        list: Relevant interactions from the database.
    """
    try:
        # Encode the query into a vector
        query_embedding = vectorizer.encode(query).tolist()
        
        # Perform a similarity search in Qdrant
        results = client.search(
            collection_name="systems",
            query_vector=query_embedding,
            limit=top_k
        )
        
        # Filter results based on the role (consumer/producer)
        filtered_results = []
        for result in results:
            if role == "consumer" and query in result.payload["consumer"]:
                filtered_results.append(result.payload)
            elif role == "producer" and query in result.payload["producer"]:
                filtered_results.append(result.payload)
        
        return filtered_results
    except Exception as e:
        print(f"Error querying vector database: {e}")
        return []

# Example Query
query = "HR"
role = "consumer"  # Can be "consumer" or "producer"
retrieved_context = query_vector_db(vector_db_client, query, role=role, top_k=5)

# Display Results
print(f"Results for query '{query}' as a {role}:")
for context in retrieved_context:
    print(context)

In [ ]:
import os
import boto3

# Load environment variables
AWS_BEDROCK_INFERENCE_PROFILE_ARN = os.getenv("AWS_BEDROCK_INFERENCE_PROFILE_ARN")
AWS_BEDROCK_AGENT_ARN = os.getenv("AWS_BEDROCK_AGENT_ARN")
AWS_REGION = os.getenv("AWS_REGION", "us-east-1")  # Default to us-east-1

# Validate environment variables
assert AWS_BEDROCK_INFERENCE_PROFILE_ARN, "AWS_BEDROCK_INFERENCE_PROFILE_ARN is not set"
assert AWS_BEDROCK_AGENT_ARN, "AWS_BEDROCK_AGENT_ARN is not set"

# Initialize Bedrock Runtime client
bedrock_client_runtime = boto3.client(
    "bedrock-runtime",
    region_name=AWS_REGION,
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY")
)

print("Bedrock Runtime client initialized successfully!")



In [ ]:
import os

# Load environment variables
AWS_REGION = os.getenv("AWS_REGION", "us-east-2")
AWS_BEDROCK_AGENT_ID = os.getenv("AWS_BEDROCK_AGENT_ID")
AWS_BEDROCK_AGENT_ALIAS_ID = os.getenv("AWS_BEDROCK_AGENT_ALIAS_ID")

# Validate environment variables
assert AWS_BEDROCK_AGENT_ID, "AWS_BEDROCK_AGENT_ID is not set"
assert AWS_BEDROCK_AGENT_ALIAS_ID, "AWS_BEDROCK_AGENT_ALIAS_ID is not set"

# Initialize Bedrock Agent Runtime client
bedrock_agent_runtime_client = boto3.client(
    service_name="bedrock-agent-runtime",
    region_name=AWS_REGION,
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY")
)

print("Bedrock Agent Runtime client initialized successfully!")

In [ ]:
import json

def invoke_agent_with_runtime(query, context):
    """
    Invoke the Bedrock agent using agent ID and alias ID.

    Args:
        query (str): The user's query.
        context (list): Contextual information for the agent.

    Returns:
        str: The response from the agent.
    """
    try:
        # Format context into a readable string
        formatted_context = "\n".join([
            f"Consumer: {item['consumer']}, Producer: {item['producer']}, Integration: {item['integration']}"
            for item in context
        ])
        
        # Construct input for the agent
        input_text = f"Query: {query}\nContext:\n{formatted_context}"
        print("Input Text Sent to Agent:")
        print(input_text)

        # Invoke the agent
        response = bedrock_agent_runtime_client.invoke_agent(
            agentId=AWS_BEDROCK_AGENT_ID,
            agentAliasId=AWS_BEDROCK_AGENT_ALIAS_ID,
            sessionId="test-session",
            inputText=input_text
        )

        # Print out the full raw response for debugging
        print("Full Raw Response from Bedrock Agent:")
        print(response)

        completion = ""

        for event in response.get("completion"):
                chunk = event["chunk"]
                completion = completion + chunk["bytes"].decode()

    except ClientError as e:
        logger.error(f"Couldn't invoke agent. {e}")
        raise

    return completion

In [ ]:
# Example query and context
test_query = "What systems does HR interact with?"
test_context = [
    {"consumer": "HR", "producer": "Payroll System", "integration": "API"},
    {"consumer": "HR", "producer": "Attendance System", "integration": "Database"}
]

# Invoke the agent
response = invoke_agent_with_runtime(test_query, test_context)

# Print the response
print("Agent Response:")
print(response)

In [ ]:
# agent instructions
# Instructions for the Agent
# You are an interactive REST API agent designed to process natural language queries about system interactions. Your primary tasks include: 1. Querying a vector database (Qdrant) for relevant system relationships based on user input. 2. Generating a JSON response that adheres to the following structure: { "nodes": [{"id": "System Name"}], "edges": [{"consumer": "System A", "producer": "System B", "integration": "Type"}] } 3. Supporting queries about relationships, such as: - "What systems interact with System X?" - "Show me the integrations for HR systems." - "List all systems that System Y is a consumer for." Rules: 1. Always return responses as a single JSON object. 2. Deduplicate nodes and edges in the response. 3. Do not include explanations or additional text outside the JSON object. If a query does not match the intended purpose, respond with an error message in JSON format: { "error": "Invalid query. Please ask about system interactions." } --- ### **Key Notes** - These instructions define the agent's role clearly. - They constrain the output to ensure compatibility with downstream consumers like D3.js. - They provide examples for users and handle invalid queries gracefully.